In [13]:
#環境設置
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd
import requests
import requests.packages.urllib3
requests.packages.urllib3.disable_warnings()
import time
driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 


In [ ]:
#新北市

    output.append(
        crawlers.primo_crawler(
            "淡江大學", "https://uco-network.primo.exlibrisgroup.com/discovery/search?query=any,contains,",
            ISBN, "&tab=Everything&search_scope=MyInst_and_CI&vid=886UCO_TKU:886TKU_INST&lang=zh-tw&offset=0", driver, 
            "neutralized-button.layout-full-width.layout-display-flex.md-button.md-ink-ripple.layout-row"
        )
    )
    output.append(
        crawlers.clickclick_crawler(
            "馬偕醫學院", "http://aleph.library.mmc.edu.tw/F?func=find-b&adjacent=Y&find_code=WRD&local_base=TOP02&request=&TY=",
            ISBN, "8", "span/a[1]", '/html/body/table[10]', driver
        )
    )

#台北市

    output.append(
        crawlers.primo_crawler(
            "臺灣大學", "https://ntu.primo.exlibrisgroup.com/discovery/search?query=any,contains,",
            ISBN, "&tab=Everything&search_scope=MyInst_and_CI&vid=886NTU_INST:886NTU_INST&offset=0", driver, "layout-align-space-between-center.layout-row.flex-100"
        )
    )

    output.append(
        crawlers.primo_crawler(
            "銘傳大學", "https://uco-mcu.primo.exlibrisgroup.com/discovery/search?query=any,contains,",
            ISBN, "&tab=Everything&search_scope=MyInst_and_CI&vid=886UCO_MCU:886MCU_INST&lang=zh-tw&offset=0", driver, "md-2-line.md-no-proxy._md"
        )
    )

    output.append(
        crawlers.primo_crawler(
            "東吳大學", "https://uco-scu.primo.exlibrisgroup.com/discovery/search?query=any,contains,",
            ISBN, "&tab=Everything&search_scope=MyInst_and_CI&vid=886UCO_SCU:886SCU_INST&lang=zh-tw&offset=0", driver, "md-2-line.md-no-proxy._md"
        )
    )

    output.append(
        crawlers.primo_crawler(
            "政治大學", "https://nccu.primo.exlibrisgroup.com/discovery/search?query=any,contains,",
            ISBN, "&tab=Everything&search_scope=MyInst_and_CI&vid=886NCCU_INST:886NCCU_INST", driver, "layout-align-space-between-center.layout-row.flex-100"
        )
    )

    output.append(
        crawlers.web2.0_crawler(
            "臺北藝術大學", "http://203.64.5.158/webpac/search/?q=",
            ISBN, "&field=isn&op=AND&type=&match=", driver
        )
    )

#桃園

    output.append(
        crawlers.primo_greendot_crawler(
            "長庚大學", "https://primo.lib.cgu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Advanced&tab=default_tab&indx=1&dum=true&srt=rank&vid=CGU&frbg=&tb=t&vl%2812508471UI0%29=isbn&vl%2812508471UI0%29=title&vl%2812508471UI0%29=isbn&vl%281UIStartWith0%29=contains&vl%28freeText0%29=",
            ISBN, "&vl%28boolOperator0%29=AND&vl%2812508474UI1%29=creator&vl%2812508474UI1%29=title&vl%2812508474UI1%29=creator&vl%281UIStartWith1%29=contains&vl%28freeText1%29=&vl%28boolOperator1%29=AND&vl%2812508470UI2%29=any&vl%2812508470UI2%29=title&vl%2812508470UI2%29=any&vl%281UIStartWith2%29=contains&vl%28freeText2%29=&vl%28boolOperator2%29=AND&vl%2812626940UI3%29=any&vl%2812626940UI3%29=title&vl%2812626940UI3%29=any&vl%281UIStartWith3%29=contains&vl%28freeText3%29=&vl%28boolOperator3%29=AND&vl%28D2240502UI4%29=all_items&vl%2853081356UI5%29=all_items&vl%28D2240500UI6%29=all_items&Submit=%E6%AA%A2%E7%B4%A2", driver
        )
    )

#新竹

    output.append(
        crawlers.primo_greendot_crawler(
            "交通大學", "https://ustcate0.lib.nctu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=ils_article&indx=1&dum=true&srt=rank&vid=NCTU&frbg=&tb=t&vl%28freeText0%29=",
            ISBN, "&scp.scps=scope%3A%28NCTU_AV%29%2Cscope%3A%28NCTU_BOOK%29%2Cscope%3A%28NCTU_DT%29%2Cscope%3A%28NCTU_JOURNAL%29%2Cscope%3A%28NCTU%29%2Cprimo_central_multiple_fe&vl%281UI0%29=contains", driver
        )
    )

    output.append(
        crawlers.clickclick_crawler(
            "工業技術研究院", "http://61.61.255.73/F?func=find-b-0",
            ISBN, "7", "a/img", '/html/body/table[10]', driver
        )
    )

#苗栗縣
    output.append(
        crawlers.web2.0_crawler(
            "國家衛生研究院", "http://webpac.nhri.edu.tw/webpac/search/?q=",
            ISBN, "&field=text&op=AND&type=&department=&college=&dep=&searchmode=&active=1&match=smart", driver
        )
    )

#台中

    output.append(
        crawlers.web2.0_crawler(
            "中山醫學大學", "http://140.128.138.208/webpac/search/?q=",
            ISBN, "&field=text&op=AND&type=&department=&active=1&match=smart", driver
        )
    )

    output.append(
        crawlers.web2.0_crawler(
            "勤益科技大學", "http://140.128.95.172/webpac/search/?q=",
            ISBN, "&field=isn&op=AND&type=&department=&college=&dep=&active=1&match=", driver
        )
    )

#嘉義縣

    output.append(
        crawlers.primo_greendot_crawler(
            "中正大學", "http://primo.lib.ccu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Advanced&tab=default_tab&indx=1&dum=true&srt=rank&vid=CCU&frbg=&tb=t&vl%28256032279UI0%29=isbn&vl%28256032279UI0%29=title&vl%28256032279UI0%29=any&vl%281UIStartWith0%29=contains&vl%28freeText0%29=",
            ISBN, "&vl%282853831UI0%29=AND&vl%28256032278UI1%29=any&vl%28256032278UI1%29=title&vl%28256032278UI1%29=any&vl%281UIStartWith1%29=contains&vl%28freeText1%29=&vl%282853829UI1%29=AND&vl%28256032320UI2%29=any&vl%28256032320UI2%29=title&vl%28256032320UI2%29=any&vl%281UIStartWith2%29=contains&vl%28freeText2%29=&vl%282853831UI2%29=AND&vl%28D2853835UI3%29=all_items&vl%28256032346UI4%29=all_items&vl%28D2853833UI5%29=all_items&Submit=%E6%AA%A2%E7%B4%A2", driver
        )
    )

#高雄市

    output.append(
        crawlers.web2.0_crawler(
            "義守大學", "http://webpac.isu.edu.tw/webpac/search/?q=%22",
            ISBN, "%22&field=text&op=AND&type=&department=&college=&dep=&order=-pyr&active=1&match=smart", driver
        )
    )
    
    output.append(
        crawlers.primo_crawler(
            "高雄科技大學", "https://nkust.primo.exlibrisgroup.com/discovery/search?query=any,contains,",
            ISBN, "&tab=Everything&search_scope=MyInst_and_CI&vid=886NKUST_INST:86NKUST&lang=zh-tw&offset=0,", driver, 
            "layout-align-space-between-center.layout-row.flex-100"
        )
    )    


In [18]:
#台大、政大、淡江、東吳：測試

my_capabilities = DesiredCapabilities.CHROME
my_capabilities["pageLoadStrategy"] = "eager"

org = "高雄餐旅大學"
ISBN = str(9789573229131)
url = "https://find.nkuht.edu.tw/primo-explore/search?query=any,contains," + ISBN + "&tab=default_tab&search_scope=%E6%9F%A5%E9%A4%A8%E8%97%8F&vid=NKUHT_N&offset=0"
tcn = "layout-align-space-between-center.layout-row.flex-100"

primo_lst = []
def primo_finding(org, tcn): #primo爬資訊的def ；#tcn = thelist_class_name
    sub_df_lst = []
    time.sleep(2)
    try:
        back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
    except:
        back = None
    if back != None:
        back.click()

    element = WebDriverWait(driver, ).until(EC.presence_of_element_located((By.CLASS_NAME, tcn)))
    thelist = driver.find_elements_by_class_name(tcn)
    if tcn == 'md-2-line.md-no-proxy._md': #如果是東吳或銘傳
        thelist = thelist[0:-2]
    else:
        pass

    for row in thelist:
        plist = row.find_elements_by_tag_name("p")
        where = row.find_elements_by_tag_name("h3")
        i = len(where)
        for sth in plist:
            a = sth.find_elements_by_tag_name("span")
            for _ in range(i):
                now_url = driver.current_url
                new_row = [org, where[_].text, a[4].text, a[0].text, now_url]
                sub_df_lst.append(new_row)
                break
            break
    return sub_df_lst

try:
    # 進入《館藏系統》頁面
    driver.get(url)
    # 等待＂進階查詢的按鈕＂直到出現：click
    time.sleep(3)

    try: #開始爬蟲
        editions = driver.find_elements_by_class_name('item-title') 
        if len(editions) > 1: #如果最外面有兩個版本(默認點進去不會再分版本了啦)(ex.政大 9789861371955)，直接交給下面處理
            pass
        else: #如果最外面只有一個版本，那有可能點進去還有再分，先click進去，再分一個版本跟多個版本的狀況
            time.sleep(2)
            editions[0].click()
            time.sleep(5)
            editions = driver.find_elements_by_class_name('item-title') #這時候是第二層的分版本了！(ex.政大 9789869109321)
            
        try: #先找叉叉確定是不是在最裡層了
            back_check = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back_check = None
        if back_check == None: #多個版本才要再跑迴圈(找不到叉叉代表不在最裡面，可知不是一個版本)
            for i in range(0, len(editions)): #有幾個版本就跑幾次，不管哪一層版本都適用
                into = editions[i].click()
                time.sleep(3)
                primo_lst += primo_finding(org, tcn)
                try: 
                    back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple").click()
                except:
                    back2 = None

        else: #如果只有一個版本(有叉叉的意思)，那前面已經click過了不能再做
            time.sleep(3)
            primo_lst += primo_finding(org, tcn)
    except:
        pass
except:
    pass
print(primo_lst)

[['高雄餐旅大學', '', '', '', 'https://find.nkuht.edu.tw/primo-explore/fulldisplay?docid=NKUHT_ALEPH005425001&context=L&vid=NKUHT_N&lang=zh_TW&search_scope=%E6%9F%A5%E9%A4%A8%E8%97%8F&adaptor=Local%20Search%20Engine&tab=default_tab&query=any,contains,9789573229131&offset=0']]


In [1]:
# web2.0  def  (館別跟館藏處要選不一樣的drop，義守有分館)

def web2.0_crawler(org, url_front, ISBN, url_behind, driver):
    url = url_front + ISBN + url_behind
    try:
        driver.get(url) 
        title = driver.find_element_by_xpath('/html/body/div/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/div[3]/div/ul/li/div/div[2]/h3/a').click()
        table_url = driver.current_url
        html_text = driver.page_source
        dfs = pd.read_html(html_text, encoding="utf-8")
        df_web2.0 = dfs[0]

        df_web2.0.insert(0, "圖書館", ["臺北藝術大學" for i in range(df_web2.0.shape[0])])
        df_web2.0.rename(columns={ "狀態": "館藏狀態"}, inplace=True)
        df_lst.append(df_web2.0)
        return pd.concat(tgt, axis=0, ignore_index=True)
    except:
        print(f"「{url}」無法爬取！")


SyntaxError: invalid decimal literal (<ipython-input-1-ffd2fedc2a42>, line 3)

In [9]:
# web2.0(北藝大，不重要)

org =  "中山醫學大學"
url_front = "http://140.128.138.208/webpac/search/?q=" 
ISBN = str(9789573317241)
url_behind = '&field=text&op=AND&type=&department=&active=1&match=smart'
url = url_front + ISBN + url_behind
df_lst = []

try:
    driver.get(url) 
    title = driver.find_element_by_xpath('/html/body/div/div[1]/div[2]/div/div/div[2]/div[3]/div[1]/div[3]/div/ul/li/div/div[2]/h3/a').click()
    time.sleep(2)
    html_text = driver.page_source
    dfs = pd.read_html(html_text, encoding="utf-8")
    df_tnua = dfs[0]

    df_tnua.insert(0, "圖書館", ["中山醫學大學" for i in range(df_tnua.shape[0])])
    for deleted in ["館別", "卷期", "館藏類型", "到期日", "條碼號"]:
        df_tnua.pop(deleted)
    df_tnua.rename(columns={ "狀態": "館藏狀態"}, inplace=True)
    df_lst.append(df_tnua)
    print(df_lst)
except:
    print(f"「{url}」無法爬取！")

[      圖書館    館藏地                  索書號 館藏狀態  Unnamed: 8
0  中山醫學大學  中文圖書區      873.57 6014 v.1  在架上         NaN
1  中山醫學大學  中文圖書區  873.57 6014 v.1/c.1  在架上         NaN]


In [16]:
#要一直點進去的(馬偕醫學院、工研院)


def clickclick_crawler(org, url ISBN, xpath_num, xpath_detail, table_xpath, driver):
    clickclick_lst = []
    ISBN_xpath = "/html/body/table[6]/tbody/tr/td[1]/form/fieldset[1]/select/option[" + xpath_num + "]"
    try:
        driver.get(url)
        use_ISBN = driver.find_element_by_xpath(ISBN_xpath).click()
        search_input = driver.find_element_by_name("y")
        search_input.send_keys(ISBN)
        gogo = driver.find_element_by_name("Search").click()
        
        time.sleep(2)
        where2 = driver.find_element_by_class_name("brieftit").click()
        time.sleep(1) 
        where3_xpath = "/html/body/table[9]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/" + xpath_detail
        where3 = driver.find_element_by_xpath(where3_xpath).click()
        time.sleep(2)
        table = driver.find_element_by_xpath(table_xpath)
        trlist = table.find_elements_by_tag_name('tr')
        for row in trlist:
            tdlist = row.find_elements_by_tag_name('td')
            for sth in tdlist:
                if org != "工業技術研究院":
                    now_url = driver.current_url
                    new_row = [org, tdlist[2].text, tdlist[4].text, tdlist[7].text, now_url]
                else:
                    new_row = [org, tdlist[2].text, tdlist[4].text, tdlist[8].text, now_url]
                clickclick_lst.append(new_row)
                break
    except:
        pass
    
    return pd.DataFrame(clickclick_lst)


9789868879348
政治大學 綜圖 目前可獲得 綜圖B1中文
政治大學 達賢館 目前可獲得 3F達賢書房


In [24]:
org =  "工業技術研究院"
url = "http://61.61.255.73/F?func=find-b-0"
ISBN = str(9789861371955)
ISBN_xpath = '/html/body/table[6]/tbody/tr/td[1]/form/fieldset[1]/select/option[7]'
xpath_detail = "a/img"
table_xpath = '/html/body/table[10]'

clickclick_lst = []
try:
    #搜尋
    driver.get(url)
    time.sleep(3)
    use_ISBN = driver.find_element_by_xpath(ISBN_xpath).click()
    search_input = driver.find_element_by_name("y")
    search_input.send_keys(ISBN)
    gogo = driver.find_element_by_name("Search").click()

    #爬蟲
    time.sleep(2)
    where2 = driver.find_element_by_class_name("brieftit").click()
    time.sleep(1) 
    where3_xpath = "/html/body/table[9]/tbody/tr/td[1]/table/tbody/tr[1]/td[2]/" + xpath_detail
    where3 = driver.find_element_by_xpath(where3_xpath).click()
    time.sleep(5)
    table = driver.find_element_by_xpath(table_xpath)
    trlist = table.find_elements_by_tag_name('tr')
    for row in trlist:
        tdlist = row.find_elements_by_tag_name('td')
        for sth in tdlist:
            if org != "工業技術研究院"
                new_row = [org, tdlist[2].text, tdlist[4].text, tdlist[7].text, url]
            else:
                new_row = [org, tdlist[2].text, tdlist[4].text, tdlist[8].text, url]
            clickclick_lst.append(new_row)
            break
except:
    pass

print(clickclick_lst)

[['工業技術研究院', 'W 158 4494', '福委會/光復勵進會圖', '', 'http://61.61.255.73/F?func=find-b-0']]


In [4]:
#台大、政大、淡江、東吳、然後銘傳沒有索書碼QQ(要另外進去但我懶🙄)

org = "高雄科技大學"
ISBN = str(9789861371955)
url = "https://nkust.primo.exlibrisgroup.com/discovery/search?query=any,contains," + ISBN + "&tab=Everything&search_scope=MyInst_and_CI&vid=886NKUST_INST:86NKUST&lang=zh-tw&offset=0,"
tcn = "md-list-item-text.layout-wrap.layout-row.flex"

def primo_crawler(org, url_front, ISBN ,url_behind , driver, tcn):
    url = url_front + ISBN + url_behind
    primo_lst = []
    def primo_finding(org, tcn): #primo爬資訊的def ；#tcn = thelist_class_name
        sub_df_lst = []
        time.sleep(2)
        try:
            back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
        except:
            back = None
        if back != None:
            back.click()

        thelist = driver.find_elements_by_class_name(tcn)
        if tcn == 'md-2-line.md-no-proxy._md': #如果是東吳或銘傳
            thelist = thelist[0:-2]
        else:
            pass

        for row in thelist:
            plist = row.find_elements_by_tag_name("p")
            where = row.find_elements_by_tag_name("h3")
            i = len(where)
            for sth in plist:
                a = sth.find_elements_by_tag_name("span")
                for _ in range(i):
                    now_url = driver.current_url
                    new_row = [org, where[_].text, a[4].text, a[0].text, now_url]
                    sub_df_lst.append(new_row)
                    break
                break
        return sub_df_lst

    try:
        # 進入《館藏系統》頁面
        driver.get(url)
        # 等待＂進階查詢的按鈕＂直到出現：click
        time.sleep(3)

        try: #開始爬蟲
            editions = driver.find_elements_by_class_name('item-title') 
            if len(editions) > 1: #如果最外面有兩個版本(默認點進去不會再分版本了啦)(ex.政大 9789861371955)，直接交給下面處理
                pass
            else: #如果最外面只有一個版本，那有可能點進去還有再分，先click進去，再分一個版本跟多個版本的狀況
                time.sleep(2)
                editions[0].click()
                time.sleep(5)
                editions = driver.find_elements_by_class_name('item-title') #這時候是第二層的分版本了！(ex.政大 9789869109321)
                
            try: #先找叉叉確定是不是在最裡層了
                back_check = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple")
            except:
                back_check = None
            if back_check == None: #多個版本才要再跑迴圈(找不到叉叉代表不在最裡面，可知不是一個版本)
                for i in range(0, len(editions)): #有幾個版本就跑幾次，不管哪一層版本都適用
                    into = editions[i].click()
                    time.sleep(3)
                    primo_lst += primo_finding(org, tcn)
                    try: 
                        back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple").click()
                    except:
                        back2 = None

            else: #如果只有一個版本(有叉叉的意思)，那前面已經click過了不能再做
                time.sleep(3)
                primo_lst += primo_finding(org, tcn)
        except:
            pass
    except:
        pass
    return pd.DataFrame(primo_lst)

In [15]:
#綠點點 def

org = "交通大學"
ISBN = str(9789573229131)
url = "https://ustcate0.lib.nctu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=ils_article&indx=1&dum=true&srt=rank&vid=NCTU&frbg=&tb=t&vl%28freeText0%29=" + ISBN + "&scp.scps=scope%3A%28NCTU_AV%29%2Cscope%3A%28NCTU_BOOK%29%2Cscope%3A%28NCTU_DT%29%2Cscope%3A%28NCTU_JOURNAL%29%2Cscope%3A%28NCTU%29%2Cprimo_central_multiple_fe&vl%281UI0%29=contains"

def primo_greendot_crawler(org, url_front, ISBN ,url_behind , driver):
    url = url_front + ISBN + url_behind
    primo_greendot_lst = []
    def primo_greendot_finding(org): #primo爬資訊的def
        sub_df_lst = []
        try:
            time.sleep(1)
            num = driver.find_elements_by_class_name('EXLLocationTableColumn1')
            status = driver.find_elements_by_class_name('EXLLocationTableColumn3')
            for i in range(0, len(num)):
                now_url = driver.current_url
                new_row = [org, "圖書館總館", num[i].text, status[i].text, now_url]
                sub_df_lst.append(new_row)
        except:
            pass
        return sub_df_lst

    
    try:
        driver.get(url)
        try: #只有一個版本
            time.sleep(2)
            place_click = driver.find_element_by_id('exlidResult0-LocationsTab').click()
            primo_greendot_lst += primo_greendot_finding(org)
        except: #有多個版本，所以要點進去再做
            time.sleep(2)
            manyeditions = driver.find_element_by_id('titleLink').click()
            time.sleep(2)
            for i in range(1, 10): #假設有十個版本吧
                try:
                    place_click2 = driver.find_element_by_id('exlidResult' + str(i) + '-LocationsTab').click()
                except:
                    continue
            primo_greendot_lst += primo_greendot_finding(org)
    except:
        pass

    return pd.DataFrame(primo_greendot_lst)

[['交通大學', '圖書館總館', '857.9 3380 85 v.5', '20210915 2359', 'https://ustcate0.lib.nctu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=ils_article&indx=1&dum=true&srt=rank&vid=NCTU&frbg=&tb=t&vl%28freeText0%29=9789573229131&scp.scps=scope%3A%28NCTU_AV%29%2Cscope%3A%28NCTU_BOOK%29%2Cscope%3A%28NCTU_DT%29%2Cscope%3A%28NCTU_JOURNAL%29%2Cscope%3A%28NCTU%29%2Cprimo_central_multiple_fe&vl%281UI0%29=contains'], ['交通大學', '圖書館總館', '857.9 3380 85 v.5', '在書架上', 'https://ustcate0.lib.nctu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Basic&tab=ils_article&indx=1&dum=true&srt=rank&vid=NCTU&frbg=&tb=t&vl%28freeText0%29=9789573229131&scp.scps=scope%3A%28NCTU_AV%29%2Cscope%3A%28NCTU_BOOK%29%2Cscope%3A%28NCTU_DT%29%2Cscope%3A%28NCTU_JOURNAL%29%2Cscope%3A%28NCTU%29%2Cprimo_central_multiple_fe&vl%281UI0%29=contains'], ['交通大學', '圖書館總館', '857.9 3380 81 v.5', '在書架上', 'https://ustcate0.lib.nctu.edu.tw/primo_library/lib

In [16]:
#綠點點測試

org = "中正大學"
ISBN = '986729193X'
url = "http://primo.lib.ccu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Advanced&tab=default_tab&indx=1&dum=true&srt=rank&vid=CCU&frbg=&tb=t&vl%28256032279UI0%29=isbn&vl%28256032279UI0%29=title&vl%28256032279UI0%29=any&vl%281UIStartWith0%29=contains&vl%28freeText0%29=" + ISBN + "&vl%282853831UI0%29=AND&vl%28256032278UI1%29=any&vl%28256032278UI1%29=title&vl%28256032278UI1%29=any&vl%281UIStartWith1%29=contains&vl%28freeText1%29=&vl%282853829UI1%29=AND&vl%28256032320UI2%29=any&vl%28256032320UI2%29=title&vl%28256032320UI2%29=any&vl%281UIStartWith2%29=contains&vl%28freeText2%29=&vl%282853831UI2%29=AND&vl%28D2853835UI3%29=all_items&vl%28256032346UI4%29=all_items&vl%28D2853833UI5%29=all_items&Submit=%E6%AA%A2%E7%B4%A2"

primo_greendot_lst = []
def primo_greendot_finding(org): #primo爬資訊的def
    sub_df_lst = []
    try:
        time.sleep(1)
        num = driver.find_elements_by_class_name('EXLLocationTableColumn1')
        status = driver.find_elements_by_class_name('EXLLocationTableColumn3')
        for i in range(0, len(num)):
            now_url = driver.current_url
            new_row = [org, "圖書館總館", num[i].text, status[i].text, now_url]
            sub_df_lst.append(new_row)
    except:
        pass
    return sub_df_lst


try:
    driver.get(url)
    try: #只有一個版本
        time.sleep(2)
        place_click = driver.find_element_by_id('exlidResult0-LocationsTab').click()
        primo_greendot_lst += primo_greendot_finding(org)
    except: #有多個版本，所以要點進去再做
        time.sleep(2)
        manyeditions = driver.find_element_by_id('titleLink').click()
        time.sleep(2)
        for i in range(1, 10): #假設有十個版本吧
            try:
                place_click2 = driver.find_element_by_id('exlidResult' + str(i) + '-LocationsTab').click()
            except:
                continue
        primo_greendot_lst += primo_greendot_finding(org)
except:
    pass

print(primo_greendot_lst)

[['中正大學', '圖書館總館', 'C05017624 / 550 8465:4 99', '在架上', 'http://primo.lib.ccu.edu.tw/primo_library/libweb/action/search.do?fn=search&ct=search&initialSearch=true&mode=Advanced&tab=default_tab&indx=1&dum=true&srt=rank&vid=CCU&frbg=&tb=t&vl%28256032279UI0%29=isbn&vl%28256032279UI0%29=title&vl%28256032279UI0%29=any&vl%281UIStartWith0%29=contains&vl%28freeText0%29=986729193X&vl%282853831UI0%29=AND&vl%28256032278UI1%29=any&vl%28256032278UI1%29=title&vl%28256032278UI1%29=any&vl%281UIStartWith1%29=contains&vl%28freeText1%29=&vl%282853829UI1%29=AND&vl%28256032320UI2%29=any&vl%28256032320UI2%29=title&vl%28256032320UI2%29=any&vl%281UIStartWith2%29=contains&vl%28freeText2%29=&vl%282853831UI2%29=AND&vl%28D2853835UI3%29=all_items&vl%28256032346UI4%29=all_items&vl%28D2853833UI5%29=all_items&Submit=%E6%AA%A2%E7%B4%A2']]


In [7]:
# 交大(舊廢)

def primo_greendot_crawler(org, url_front, ISBN ,url_behind , driver):
    url = url_front + ISBN + url_behind
    primo_greendot_lst = []
    def primo_greendot_finding(uni_name): #primo爬資訊的def
        sub_df_lst = []
        try:
            time.sleep(1)
            place = driver.find_element_by_class_name('EXLAvailabilityLibraryName')
            status = driver.find_element_by_id('rtaReturnedStatusTOP010054897964801549636893389809149347790')
            num = driver.find_element_by_xpath('//html/body/div[1]/div[3]/div[1]/div[3]/table/tbody/tr/td[4]/div[5]/form/div[2]/div[2]/div/span[2]/cite')
            new_row = [uni_name, place.text, status.text, num.text, url]
            sub_df_lst.append(new_row)
        except:
            print(QQ)
        return sub_df_lst

    driver.get(url)
    try:
        try: #只有一個版本
            time.sleep(2)
            place_click = driver.find_element_by_id('exlidResult0-LocationsTab').click()
            primo_greendot_lst += primo_greendot_finding(org)
        except: #有多個版本，所以要點進去再做
            time.sleep(2)
            manyeditions = driver.find_element_by_class_name('EXLBriefResultsDisplayMultipleLink')    
            for i in range(1, 10): #假設有十個版本吧
                try:
                    editions = driver.find_element_by_xpath('//html/body/div[1]/div[3]/div[1]/div[3]/table/tbody/tr[' + str(i) + ']/td[4]/div[2]/div/ul/li[2]/a').click()
                except:
                    pass
                primo_greendot_lst += primo_greendot_finding(org)
    except:
        print(None)
return pd.DataFrame(primo_greendot_lst)

In [17]:
# 銘傳
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://uco-mcu.primo.exlibrisgroup.com/discovery/search?vid=886UCO_MCU:886MCU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
for row in thelist[:-2]:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(1): 
            print('銘傳大學', where[_].text, a[0].text, end = "\n")
            break
        break
driver.close()

9789869109321
銘傳大學 桃園館 有可用館藏
銘傳大學 臺北館 有可用館藏


In [18]:
# 東吳
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://uco-scu.primo.exlibrisgroup.com/discovery/search?vid=886UCO_SCU:886SCU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("md-2-line.md-no-proxy._md")
for row in thelist[:-2]:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(1): 
            print('東吳大學', where[_].text, a[0].text, end = "\n")
            break
        break 
driver.close()

9789869109321
東吳大學 城區分館書庫 目前可獲得
東吳大學 城區分館書庫 目前可獲得
東吳大學 雙溪中正書庫 目前可獲得


In [21]:
# 淡江
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://uco-network.primo.exlibrisgroup.com/discovery/search?vid=886UCO_TKU:886TKU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_element_by_class_name('item-title').click()

time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("neutralized-button.layout-full-width.layout-display-flex.md-button.md-ink-ripple.layout-row")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('淡江大學', where[_].text, a[0].text, end = "\n")
            break
driver.close()

9789869109321
淡江大學 總館 可獲得
淡江大學 蘭陽校園圖書館 可獲得


In [33]:
# 政大
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\\88698\\OneDrive\\文件\\chromedriver") 
driver.get("https://nccu.primo.exlibrisgroup.com/discovery/search?vid=886NCCU_INST:886NCCU_INST&lang=zh-tw") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('searchBar')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_class_name('zero-margin.button-confirm.large-icon.md-button.md-primoExplore-theme.md-ink-ripple').click()
time.sleep(2)
title = driver.find_elements_by_class_name('item-title')[0].click()
time.sleep(1)
firstbook = driver.find_elements_by_class_name('item-title')[0].click()
time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('政治大學', where[_].text, a[0].text, a[2].text, end = "\n")
            break
        break
try:
    back2 = driver.find_element_by_class_name("md-icon-button.close-button.full-view-navigation.md-button.md-primoExplore-theme.md-ink-ripple").click()
except:
    back2 = None
secondbook = driver.find_elements_by_class_name('item-title')[1].click()
time.sleep(3)
try:
    back = driver.find_element_by_css_selector(".tab-header .back-button.button-with-icon.zero-margin.md-button.md-primoExplore-theme.md-ink-ripple")
except:
    back = None
if back != None:
    back.click()
thelist = driver.find_elements_by_class_name("layout-align-space-between-center.layout-row.flex-100")
for row in thelist:
    plist = row.find_elements_by_tag_name("p")
    where = row.find_elements_by_tag_name("h3")
    i = len(where)
    for sth in plist:
        a = sth.find_elements_by_tag_name("span")
        for _ in range(i): 
            print('政治大學', where[_].text, a[0].text, a[2].text, end = "\n")
            break
        break
    

9789869109321
政治大學 綜圖 不在圖書館內 綜圖B1中文
政治大學 達賢館 不在圖書館內 3F達賢書房
政治大學 綜圖 目前可獲得 綜圖B1中文


In [13]:
# 桃園
from selenium import webdriver # 先下載 webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\mayda\Downloads\chromedriver") 
driver.get("https://webpac.typl.gov.tw/advanceSearch")
time.sleep(5)
ISBN = input()
element = driver.find_element_by_name('searchInput')
element.send_keys(ISBN)
select = Select(driver.find_element_by_name('searchField'))
select.select_by_value("ISBN")
search_gogogo = driver.find_element_by_xpath('//html/body/div/div[1]/div/div/div/div/div/div[2]/div/div/form/div[1]/div/input[2]').click()
time.sleep(10)

try:
    show_more = driver.find_element_by_link_text("載入更多").click()
except:
    show_more = None
    
time.sleep(5)  
firstbook = driver.find_element_by_xpath('/html/body/div/div[1]/div[2]/div/div/div[2]/div/div[4]/form/div/div[2]/div[2]/div[1]/h2/a').click()
time.sleep(2) 
table = driver.find_element_by_class_name('bookplace_list')
trlist = table.find_elements_by_tag_name('tr')
for row in trlist:
    tdlist = row.find_elements_by_tag_name('td')
    for sth in tdlist:
        print('桃園市立圖書館', tdlist[0].text, tdlist[3].text, end = "\n")
        break
backtolist = driver.find_element_by_link_text('查詢列表頁').click()
time.sleep(5)
secondbook = driver.find_element_by_xpath('/html/body/div/div[1]/div/div/div/div[2]/div/div[4]/form/div/div[3]/div[2]/div[1]/h2/a').click()
time.sleep(2) 
table = driver.find_element_by_class_name('bookplace_list')
trlist = table.find_elements_by_tag_name('tr')
for row in trlist:
    tdlist = row.find_elements_by_tag_name('td')
    for sth in tdlist:
        print('桃園市立圖書館', tdlist[0].text, tdlist[3].text, end = "\n")
        break
backtolist = driver.find_element_by_link_text('查詢列表頁').click()
driver.close()

桃園市立圖書館 楊梅分館/閱覽區 已被外借/2021-08-12
桃園市立圖書館 總館/桃園分區資源中心 書在館
桃園市立圖書館 八德分館/八德 書在館
桃園市立圖書館 觀音分館/閱覽室 預約待取
桃園市立圖書館 蘆竹分館/3樓書庫 書在館
桃園市立圖書館 龍潭分館/1F開架閱覽區 書在館
桃園市立圖書館 埔心分館/一般書庫 書在館
桃園市立圖書館 龍岡分館/書庫 書在館
桃園市立圖書館 龍岡分館/書庫 書在館
桃園市立圖書館 中壢分館/書庫 書在館
桃園市立圖書館 大林分館/閱覽室 書在館


In [18]:
#基隆市
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome("C:\\Users\mayda\Downloads\chromedriver") 
driver.get("https://webpac.klccab.gov.tw/webpac/search.cfm") # 更改網址以前往不同網頁
ISBN = input()
element = driver.find_element_by_id('ss_keyword')
element.send_keys(ISBN)
search_gogogo = driver.find_element_by_name('searchBtn').click()
time.sleep(2)
table = driver.find_element_by_class_name('list.list_border')
trlist = table.find_elements_by_tag_name('tr')
for row in trlist[1:]:
    tdlist = row.find_elements_by_tag_name('td')
    for sth in tdlist:
        print('基隆市立圖書館', tdlist[1].text, tdlist[3].text, end = "\n")
        break
    
driver.close()

9789869109321
基隆市立圖書館 館藏地 館藏狀態
